Copyright 2024 shins777@gmail.com

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

## Vertex AI Search - REST API 호출로 검색

Feedback : shins777@gmail.com

이 Colab 은 대부분의 고객의 요구사항이면서, 많은 유즈케이스에서의 근간이 되는 지식검색에 대한 예제입니다.
Gronding service 로 Vertex AI Search와 Langchain 을 연동해서 정보를 검색하고 해당 검색된 정보를 Gemini 를 통해서 추론해서 답해주는 형태의 예제입니다.

* Vertex AI Search Manual references
    * https://cloud.google.com/enterprise-search?hl=ko
    * https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search


이 소스에서 지식 검색의 대상은 "정보통신 진흥 및 융합 활성화 등에 관한 특별법" 이며 해당 특별법은 아래 사이트에서 다운받았습니다.
https://www.law.go.kr/

#라이브러리 설치
*   Langchain library : https://github.com/langchain-ai/langchain
*   Langchain Vertex AI API : https://api.python.langchain.com/en/stable/google_vertexai_api_reference.html



In [1]:
%pip install --upgrade --quiet langchain langchain-core langchain-google-vertexai google-cloud-discoveryengine

Note: you may need to restart the kernel to use updated packages.


In [2]:
from IPython.display import display, Markdown

### GCP 사용자 인증 / 환경설정

GCP 인증방법은 아래와 URL 정보를 참고하여 GCP에 접근 하는 환경을 구성해야 합니다. 
* https://cloud.google.com/docs/authentication?hl=ko
* 자세한 정보는 [README.md](https://github.com/shins777/google_gen_ai_sample/blob/main/notebook/gemini/README.md) 파일 참고하세요.

In [3]:
#  아래 코드는 Colab 환경에서만 실행해주세요. 다른 환경에서는 동작하지 않습니다.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

### GCP 프로젝트 및 리전 설정
본인의 GCP 환경에 맞게 아래 설정을 구성하세요.  
* 구글의 최신버전인 gemini pro 사용을 권고드립니다.   
* 만일, 기본 버전 text bison 을 사용하려한다면, 참조하는 class 가 다르므로 주의하세요.  
* 현재 Gemini는 한국리전(asia-northeast3)을 통해서 접근이 가능합니다.
* 아래 SEARCH_URL Vertex AI Search를 구성 후 Integration 메뉴항목에서 추출된 값입니다. 

In [4]:
PROJECT_ID="ai-hangsik"
REGION="asia-northeast3"
MODEL = "gemini-1.5-pro-preview-0409"
SEARCH_URL = "https://discoveryengine.googleapis.com/v1alpha/projects/721521243942/locations/global/collections/default_collection/dataStores/it-laws-ds_1713063479348/servingConfigs/default_search:search"

In [5]:
import vertexai
import google
import google.oauth2.credentials
from google.auth import compute_engine
import google.auth.transport.requests
import requests
import json
import os

vertexai.init(project=PROJECT_ID, location=REGION)

stream = os.popen('gcloud auth print-access-token')
credential_token = stream.read().strip()


ERROR: (gcloud.auth.print-access-token) You do not currently have an active account selected.
Please run:

  $ gcloud auth login

to obtain new credentials.

If you have already logged in with a different account, run:

  $ gcloud config set account ACCOUNT

to select an already authenticated account to use.


### Vertex AI Search 검색

아래 코드는 REST API를 통해서 Verext AI Search에 직접 검색 쿼리를 보내는 예제입니다.
검색에 활용되는 패러미터는 ContentSearchSpec 에 따릅니다. 아래 정보 참고하세요.
* https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec

특히 ExtractiveContentSpec은 검색 컨텐츠를 좀더 상세하게 얻을수 있는 설정입니다. 아래 정보 참고하세요.
* https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1beta.types.SearchRequest.ContentSearchSpec.ExtractiveContentSpec

In [6]:
def retrieve_vertex_ai_search(question:str, search_url:str, page_size:int)->str:

  """ retrieve information from enterprise search ( discovery engine )"""

  # Create a credentials token to call a REST API
  headers = {
      "Authorization": "Bearer "+ credential_token,
      "Content-Type": "application/json"
  }

  query_dic ={
      "query": question,
      "page_size": str(page_size),
      "offset": 0,
      "contentSearchSpec":{

            "searchResultMode" : "CHUNKS",
            "chunkSpec" : {
                "numPreviousChunks" : 2,
                "numNextChunks" : 2
            }              
      },
  }

  data = json.dumps(query_dic)

  # Encode data as UTF8
  data=data.encode("utf8")

  response = requests.post(search_url,headers=headers, data=data)

  print(response.text)
  return response.text


#### Search 결과 파싱 로직
아래 로직은 검색된 결과를 파싱하는 로직입니다.

In [7]:
def parse_chunks(response_text:str)->dict:

    """Parse response to build a conext to be sent to LLM"""

    dict_results = json.loads(response_text)

    index = 0
    search_results = {}

    if dict_results.get('results'):

        for result in dict_results['results']:

            item = {}

            chunk = result['chunk']

            #print(chunk)
            # print(f"content {chunk['content']}")

            # print(f"documentMetadata {chunk['documentMetadata']}")
            # print(f"documentMetadata {chunk['documentMetadata']['title']}")
            # print(f"documentMetadata {chunk['documentMetadata']['uri']}")

            # print(f"documentMetadata {chunk['pageSpan']['pageStart']}")
            # print(f"documentMetadata {chunk['pageSpan']['pageEnd']}")

            # print(f"previousChunks {chunk['chunkMetadata']['previousChunks'][0]['content']}")
            # print(f"nextChunks {chunk['chunkMetadata']['nextChunks'][0]['content']}")

            item['title'] = chunk['documentMetadata']['title']
            item['uri'] = chunk['documentMetadata']['uri']
            item['pageSpan'] = f"{chunk['pageSpan']['pageStart']} ~ {chunk['pageSpan']['pageEnd']}"
            item['content'] = chunk['content']

            # chunk 는 현재 Contents에 가까운것 부터 나타남.
            p_chunks = chunk['chunkMetadata']['previousChunks']
            for p_chunk in p_chunks:
                item['content'] = p_chunk['content'] +"\n"+ item['content']

            n_chunks = chunk['chunkMetadata']['nextChunks']
            for n_chunk in n_chunks:
                item['content'] = item['content'] +"\n"+ n_chunk['content']

            search_results[f'results-{index}'] = item
            index = index+1

    return search_results

In [8]:
question = "개인정보 보호법에 대해서 설명해주세요."

page_size = 2

searched_ctx = retrieve_vertex_ai_search(question, SEARCH_URL, page_size)


{
  "error": {
    "code": 401,
    "message": "Request is missing required authentication credential. Expected OAuth 2 access token, login cookie or other valid authentication credential. See https://developers.google.com/identity/sign-in/web/devconsole-project.",
    "status": "UNAUTHENTICATED",
    "details": [
      {
        "@type": "type.googleapis.com/google.rpc.ErrorInfo",
        "reason": "CREDENTIALS_MISSING",
        "domain": "googleapis.com",
        "metadata": {
          "method": "google.cloud.discoveryengine.v1alpha.SearchService.Search",
          "service": "discoveryengine.googleapis.com"
        }
      }
    ]
  }
}



In [9]:
context = parse_chunks(searched_ctx)

print(context)

{}


### Gemini Pro 실행 - Vertex AI Search as a Grounding Service

*   VertexAI API : https://api.python.langchain.com/en/stable/llms/langchain_google_vertexai.llms.VertexAI.html#langchain_google_vertexai.llms.VertexAI

In [10]:
from langchain_google_vertexai.llms import VertexAI

gemini_pro = VertexAI( model_name = MODEL,
                  project=PROJECT_ID,
                  location=REGION,
                  verbose=True,
                  streaming=False,
                  temperature = 0.2,
                  top_p = 1,
                  top_k = 40
                 )

In [11]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain



prompt = PromptTemplate.from_template("""

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {context}
  Question : {question}

  """)

prompt = prompt.format(context=context,
                       question=question)

print(f"Prompt : {prompt}")

response = gemini_pro.invoke(prompt)
display(Markdown(response))

Prompt : 

  당신은 법률을 상담해주는 AI 어시스턴트입니다.
  아래 Question 에 대해서 반드시 Context에 있는 개별 내용을 기반으로 단계적으로 추론해서 근거를 설명하고 답변해주세요.
  Context : {}
  Question : 개인정보 보호법에 대해서 설명해주세요.

  


Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: string indices must be integers.
Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: string indices must be integers.
Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: string indices must be integers.
Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 8.0 seconds as it raised ServiceUnavailable: 503 Getting metadata from plugin failed with error: string indices must be integers.
Retrying langchain_google_vertexai.llms._completion_with_retry.<

ServiceUnavailable: 503 Getting metadata from plugin failed with error: string indices must be integers